# Tax Appeal
### Created by: Ryan Jinnette

#### This file uses selenium and pandas to scrape county website for tax records that will be used for analysis to fair price valuation

## Set Up and Imports

In [77]:
# !pip install selenium
# !pip install pandas

import pandas as pd
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_autoinstaller
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

#import timeit

## Constants and Class Definitions

In [95]:
lots = []

class lot:
    '''This class provides a framework to store data more easily instead of multiple lists.
    I can reference just the instance of the class'''
    def __init__(self, prop_id=None, geo_id=None, name=None, appraised=None, land_value=None, land_size=None, improved_value=None, sqft=None, year=None, bldg_class=None):
        self.prop_id = prop_id
        self.geo_id = geo_id
        self.name = name
        self.appraised = appraised
        self.land_value = land_value
        self.land_size = land_size
        self.improved_value = improved_value
        self.imp_sqft = sqft
        self.year = year
        self.bldg_class = bldg_class
        self.dollar_sqft = None
    
    def set_dtypes(self):
        '''used to keep things consistent so that math can be done on all cells and not throw errors'''
        print("nothing for now")
    
    def calc_sqft(self):
        return round(self.improved_value / self.sqft,2)
    

In [79]:
web_url = r'https://esearch.galvestoncad.org/Search/Result?keywords=Johnson%20Crawford#'


## Data Collection
#### Iterates over the webpage and grabs all necessary data, adds to the class and uses the XPATH of certain items to track them on the page for easy identification and repeatability

In [96]:
def collect(web_url):
    driver = webdriver.Chrome()
    driver.get(web_url)
    driver.implicitly_wait(5) # dynamic waiting to see when any certain searched for object becomes available
    time.sleep(2)
    
    #change the list view to 100 instead of default 25
    btn = driver.find_element(By.ID, 'btnFilterByPage')
    btn.click()
    
    btn = driver.find_element(By.XPATH, '/html/body/div[4]/div[3]/div/span[3]/ul/li[3]')
    btn.click()

    rows = driver.find_elements(By.XPATH, '/html/body/div[4]/div[4]/div[2]/table/tbody/tr')
    

    for i in range(2,4):
        
        # this grabs all initially avail info, will have to go into each property next
        prop_id = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]/td[1]').text
        geo_id = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]/td[3]').text
        name = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]/td[6]').text
        appraised = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]/td[10]').text
        
        #go into each property and get more details
        click_property = driver.find_element(By.XPATH, f'/html/body/div[4]/div[4]/div[2]/table/tbody/tr[{i}]')
        click_property.click()
        
        land_size = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[5]/div[2]/table/tbody/tr[2]/td[4]').text
        land_value = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[5]/div[2]/table/tbody/tr[2]/td[7]').text
        # in case any of the following don't exist. IE the lot is land only
        try:
            improvement = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[2]/div[2]/div/table/tbody/tr[3]/td').text
        except NoSuchElementException:
            improvement = 0

        try:
            imp_sqft = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[4]/div[2]/div[1]/span[3]').text
        except NoSuchElementException:
            imp_sqft = 0

        try:
            year = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[4]/div[2]/table/tbody/tr[2]/td[5]').text
        except NoSuchElementException:
            year = 'Nan'

        try:
            bldg_class = driver.find_element(By.XPATH, f'/html/body/div[4]/div[1]/div[4]/div[2]/table/tbody/tr[2]/td[3]').text
        except NoSuchElementException:
            bldg_class = 'Nan'

        lot_info = lot(prop_id, geo_id, name, appraised, land_value,land_size, improvement, imp_sqft, year, bldg_class)
        lots.append(lot_info)
        driver.back()
        
        
collect(web_url)




In [98]:
lot = lots[0]
attr = vars(lot).keys()
#print(attr)
for att in attr:
    print(getattr(lot,att))

221853
4275-0000-0001-000
CASON GREGORY A & JONICA A
$289,040
$289,040
17,986.00
$0 (+)
0
Nan
Nan
None
